Entramos al apartado de limpieza y preparación de SQL

In [7]:
!pip install scikit-learn

In [8]:
!pip install plotly

In [9]:
import sys
print(sys.executable)

/usr/local/bin/python3


In [10]:
pip uninstall scikit-learn

Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Would remove:
    /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scikit_learn-1.5.2.dist-info/*
    /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip uninstall plotly

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install plotly


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install nbformat

  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached fastjsonschema-2.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached referencing-0.35.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.20.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.2 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached fastjsonschema-2.20.0-py3-none-any.whl (23 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
Using cached referencing-0.35.1-py3-none-any.whl (26 kB)
Using cached rpds_py-0.20.0-cp312-cp312-macosx_11_0_arm64.whl (313 kB)

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [70]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import requests
import sqlite3
import re
from sklearn.cluster import KMeans
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import nbformat
# Lee el archivo CSV
df = pd.read_csv('portales_empleo.csv')

# Muestra las primeras filas del DataFrame
print(df.head())

df = df.drop(columns=['Unnamed: 0'])
# Eliminar filas duplicadas en su totalidad
df = df.drop_duplicates()


def limpieza_salario(rango_salarios):
    # Eliminar 'EUR' y comas
    rango_salarios = rango_salarios.replace('EUR', '').replace(',', '').strip()
    
    # Separar en salario mínimo y máximo
    if ' - ' in rango_salarios:
        rango = rango_salarios.split(' - ')
        salario_min = int(rango[0].strip())
        salario_max = int(rango[1].strip())
    else:
        # Si no hay rango, usar el valor como salario mínimo y máximo
        salario_min = salario_max = int(rango_salarios)

    return salario_min, salario_max

# Aplicar la función al DataFrame
df['sueldo_min'], df['sueldo_max'] = zip(*df['Sueldo'].map(limpieza_salario))

# Elimina el texto entre paréntesis en la columna 'título'
df['Título'] = df['Título'].str.replace(r'\(.*?\)', '', regex=True)

# Elimina los signos '|' y '/'
df['Título'] = df['Título'].str.replace(r'[|/]', '', regex=True)

# Elimina espacios en blanco adicionales
df['Título'] = df['Título'].str.strip()

# Elimina el texto entre corchetes en la columna 'título'
df['Título'] = df['Título'].str.replace(r'\[.*?\]', '', regex=True)

# Elimina los signos '|', '/' y '-'
df['Título'] = df['Título'].str.replace(r'[|/-]', '', regex=True)

# Elimina espacios en blanco adicionales
df['Título'] = df['Título'].str.strip()

# Muestra las primeras filas del DataFrame
df.head(5)
# Función para actualizar títulos
def actualizar_titulo(titulo):
    # Reemplazar 'jr' por 'Junior' y 'sr' por 'Senior'
    titulo = titulo.replace(' jr ', ' Junior ').replace(' sr ', ' Senior ')
    titulo = titulo.replace('jr ', 'Junior ').replace('sr ', 'Senior ')
    titulo = titulo.replace(' jr', ' Junior').replace(' sr', ' Senior')
    return titulo

# Aplicar la función de actualización a la columna 'Título'
df['Título'] = df['Título'].apply(actualizar_titulo)
def get_expertise(title):
    if 'Senior' in title or 'senior' in title or 'master' in title or 'manager' in title:
        return 'Senior'
    elif 'Junior' in title or 'junior' in title:
        return 'Junior'
    elif 'intern' in title or 'becario' in title:
        return 'Intern'
    elif 'Lead' in title or 'lead' in title:
        return 'Lead' 
    else:
        return 'Regular'

# Crear la nueva columna "Expertise" aplicando la función sobre la columna "Título"
df['Expertise'] = df['Título'].apply(get_expertise)
import re
# Listas de palabras clave (nombres de empresas y sectores)
empresas = ['santander', 'amazon', 'microsoft', 'vie', 'bcg', 'mckinsey']  # Añadir más nombres de empresas
sectores = ['marketing', 'medtech', 'fintech', 'ecommerce', 'finance', 'game','genai','digital', 'tax', 'governance', 'energy' ]  # Añadir más sectores

# Convertir las listas a un conjunto de expresiones regulares para facilitar la búsqueda
regex_empresas = re.compile(r'\b(' + '|'.join(empresas) + r')\b', re.IGNORECASE)
regex_sectores = re.compile(r'\b(' + '|'.join(sectores) + r')\b', re.IGNORECASE)

# Función para extraer empresa
def extraer_empresa(titulo):
    match = regex_empresas.search(titulo)
    return match.group(0) if match else None

# Función para extraer sector
def extraer_sector(titulo):
    match = regex_sectores.search(titulo)
    return match.group(0) if match else None

# Función para limpiar el título
def limpiar_titulo(titulo):
    # Eliminar nombres de empresas y sectores del título
    titulo = regex_empresas.sub('', titulo)
    titulo = regex_sectores.sub('', titulo)
    # Eliminar espacios extra
    return ' '.join(titulo.split())

# Aplicar las funciones a las columnas del DataFrame
df['Empresa'] = df['Título'].apply(extraer_empresa)
df['Sector'] = df['Título'].apply(extraer_sector)
df['Título'] = df['Título'].apply(limpiar_titulo)
df.head(12)
# Detecta la palabra clave "Inglés" y asigna a la nueva columna 'Otro Idioma'
df['Otro Idioma'] = df['Título'].apply(lambda x: 'inglés' if 'inglés' in x else None)

# Detecta la palabra clave "Inglés" y asigna a la nueva columna 'Otro Idioma'
df['Otro Idioma'] = df['Título'].apply(lambda x: 'English' if 'English' in x else None)

# Detecta las palabras clave "Híbrido", "Remoto" o "Presencial" y asigna a la nueva columna 'Modalidad'
df['Modalidad'] = df['Título'].apply(lambda x: 'Híbrido' if 'Híbrido' in x else ('Remoto' if 'Remoto' in x else ('Presencial' if 'Presencial' in x else None)))

df['Otro Idioma'] = df['Título'].apply(lambda x: 'english' if 'english' in x else None)
df['Otro Idioma'] = df['Título'].apply(lambda x: 'french' if 'french' in x else None)
df['Otro Idioma'] = df['Título'].apply(lambda x: 'francés' if 'francés' in x else None)
# Función para detectar los idiomas
def detectar_idiomas(titulo):
    if 'inglés' in titulo.lower() or 'english' in titulo.lower():
        return 'Inglés'
    elif 'francés' in titulo.lower() or 'french' in titulo.lower():
        return 'Francés'
    # Puedes añadir más idiomas aquí
    return None

# Aplicar la función para detectar idiomas
df['Otro Idioma'] = df['Título'].apply(detectar_idiomas)
titulos_agrupados = df['Otro Idioma'].value_counts()
print(titulos_agrupados)
# Diccionario de ubicaciones y sus modificaciones
ubicaciones_dict = {
    'Boadilla del Monte': 'Madrid',
    'Alcobendas': 'Madrid',
    'Leganés': 'Madrid',
    'Getafe': 'Madrid',
    'Barcelona, Centro': 'Cataluña',
    'Esplugues de Llobregat': 'Cataluña',
    'Barcelona': 'Cataluña',
    'Pozuelo de Alarcón': 'Madrid',
    'Tres Cantos': 'Madrid',
    'Villaviciosa de Odón': 'Madrid',
    'Sant Just Desvern': 'Cataluña',
    'Sant Cugat del Vallès':'Cataluña'
}

# Función para reemplazar la ubicación según el diccionario
df['Ubicación'] = df['Ubicación'].replace(ubicaciones_dict)
titulos_agrupados = df['Ubicación'].value_counts()
print(titulos_agrupados)
titulos_agrupados = df['Empresa'].value_counts()
print(titulos_agrupados)
# Remover caracteres innecesarios y estandarizar
df['Título'] = df['Título'].str.replace(r'[^\w\s]', '', regex=True).str.strip().str.lower()
# Crear columna de 'Puesto' principal basado en palabras clave
def clasificar_rol(titulo):
    if 'data scientist' in titulo:
        return 'Data Scientist'
    elif 'data applied scientist ii at barcelona spain' in titulo:
        return 'Data Scientist'
    elif 'data engineer' in titulo:
        return 'Data Engineer'
    elif 'analyst' in titulo:
        return 'Data Analyst'
    elif 'machine learning' in titulo:
        return 'Machine Learning Engineer'
    elif 'business intelligence' in titulo:
        return 'Business Intelligence'
    elif 'consultant' in titulo:
        return 'Consultant'
    elif 'specialist' in titulo:
        return 'Data Specialist'
    else:
        return 'Other'

df['Puesto'] = df['Título'].apply(clasificar_rol)
# Reordenar columnas en el orden especificado
columnas_ordenadas = [
    'Título', 'Puesto', 'sueldo_min','sueldo_max','Expertise', 'Empresa', 'Modalidad', 'Sector', 
    'Ubicación', 'Descripción', 'Servicios', 'Otro Idioma'
]
df = df[columnas_ordenadas]
# Función para clasificar el puesto basado en palabras clave extendidas
def clasificar_rol(titulo):
    titulo = titulo.lower()
    if 'data scientist' in titulo:
        return 'Data Scientist'
    elif 'data applied scientist ii at barcelona spain' in titulo:
        return 'Data Scientist'
    elif 'data engineer' in titulo:
        return 'Data Engineer'
    elif 'data manager' in titulo or 'manager data' in titulo:
        return 'Data Manager'
    elif 'head of data' in titulo:
        return 'Data Manager'
    elif 'data expert' in titulo:
        return 'Data Expert'
    elif 'big data architect' in titulo:
        return 'Big Data Architect'
    elif 'manager data cloud' in titulo:
        return 'Data Manager Cloud'
    elif 'data analytics' in titulo:
        return 'Data Analytics'
    elif 'ai innovation architect' in titulo:
        return 'AI Innovation Architect'
    elif 'data management engineer' in titulo:
        return 'Data Manager'
    elif 'business intelligence' in titulo:
        return 'Business Intelligence'
    elif 'analyst' in titulo:
        return 'Data Analyst'
    elif 'consultant' in titulo:
        return 'Consultant'
    elif 'machine learning' in titulo:
        return 'Machine Learning Engineer'
    elif 'specialist' in titulo:
        return 'Data Specialist'
    elif 'data developer' in titulo:
        return 'Data Developer'
    elif 'project manager' in titulo:
        return 'Project Manager'
    else:
        return 'Other'

# Aplicar la función a la columna "Título" para crear o actualizar la columna "Puesto"
df['Puesto'] = df['Título'].apply(clasificar_rol)

# Función para limpiar nombres de ciudades y códigos de puesto directamente en "Título"
def limpiar_titulo(titulo):
    # Eliminación de nombres de ciudades comunes en títulos
    ciudades = ["barcelone", "alicante", "catalonia", "madrid", "bilbao"]  # Añadir otras ciudades según sea necesario
    for ciudad in ciudades:
        titulo = re.sub(rf"\b{ciudad}\b", "", titulo, flags=re.IGNORECASE)
    
    # Eliminación de códigos de puesto que suelen ser letras y números juntos
    titulo = re.sub(r"\b[a-zA-Z]{2,3}\d{3}\b", "", titulo)
    
    # Eliminación de espacios extras generados por las sustituciones
    titulo = re.sub(r"\s+", " ", titulo).strip()
    return titulo

# Aplicar la función de limpieza a la columna "Título" para modificarla directamente
df['Título'] = df['Título'].apply(limpiar_titulo)

# Función para extraer herramientas tecnológicas
def extraer_herramientas(titulo):
    herramientas = ["Salesforce", "CRM", "Power BI", "Qlik", "Python", "Tableau", "SAS", "SAP", "BigQuery", "Datastage", "Analytics", "Pentaho", "Spark"]  # Añadir otras herramientas según sea necesario
    herramientas_encontradas = [herramienta for herramienta in herramientas if herramienta.lower() in titulo.lower()]
    return ", ".join(herramientas_encontradas) if herramientas_encontradas else None

# Crear la columna "Herramientas" con las herramientas tecnológicas encontradas en el título
df['EntornoTEC'] = df['Título'].apply(extraer_herramientas)
def convertir_abreviaciones(titulo):
    titulo = titulo.lower()  # Convertir a minúsculas para comparación
    if 'sr' in titulo:
        titulo = titulo.replace('sr', 'senior')
    if 'jr' in titulo:
        titulo = titulo.replace('jr', 'junior')
    return titulo

# Aplicar la función para convertir los títulos
df['Título'] = df['Título'].apply(convertir_abreviaciones)
def get_expertise(title):
    if 'Senior' in title or 'senior' in title or 'master' in title or 'manager' in title:
        return 'Senior'
    elif 'Junior' in title or 'junior' in title or 'associate' in title:
        return 'Junior'
    elif 'intern' in title or 'becario' in title or 'becarioa' in title:
        return 'Intern'
    elif 'Lead' in title or 'lead' in title or 'head' in title:
        return 'Lead' 
    else:
        return 'Regular'

# Crear la nueva columna "Expertise" aplicando la función sobre la columna "Título"
df['Expertise'] = df['Título'].apply(get_expertise)
titulos_agrupados = df['Puesto'].value_counts()
# Filtrar filas que contengan el texto deseado en la columna "Título"
filtro = df[df['Puesto'].str.contains("Other", case=False, na=False)]

# Función para agregar espacios entre palabras correctamente
def add_spaces(text):
    # Añade un espacio entre palabras que estén pegadas (ej: "SnacksWeekly" -> "Snacks Weekly")
    corrected_text = re.sub(r'(?<=[a-zA-Z])(?=[A-Z])', ' ', text)
    return corrected_text

# Aplicar la función a la columna "Servicios"
df['Servicios'] = df['Servicios'].apply(add_spaces)
# Lista de palabras comunes a eliminar
common_words = r'\b(de|a|in|or|and|the|of|en|con|para|por|y|del|la|los|las|un|una)\b'

# Definición de las palabras clave para cada categoría
benefits_keywords = r'\b(Tarjeta de restaurante|Teletrabajo opcional|Flexible hours|Modern office environment|Continuous learning programs|Mentoring programs|Collaborative culture|Oportunidades de crecimiento profesional|Paquete competitivo de compensación y beneficios|Retirement program|Well-being programs|Diversity initiatives|Work-life balance|Remote work)\b'
skills_keywords = r'\b(Neural networks|Bayesian modeling|Data storytelling|Análisis de datos|Métodos estadísticos|Business Intelligence|Data Science|Machine Learning|Deep Learning|Data Visualization|Problem Solving|Data Analysis|Reporting|Data governance|Statistical Analysis|Predictive Modeling)\b'
tools_keywords = r'\b(Python|R|Pyspark|Power BI|Tableau|Databricks|GCP|SAS|SQL|Excel|VBA|Google Analytics|Adobe Analytics|Domo|Snowflake|TensorFlow|Keras|Scikit-learn|Power Platforms|Jupyter Notebooks|Azure|AWS|Docker|GitHub|Streamlit|Pytorch|BigQuery|SQL)\b'
education_keywords = r'\b(Degree in Computing or Statistics|Bachelor\'s Degree|Master\'s Degree|PhD|Licenciatura|Grado|Engineering Degree|Data Science Degree|Statistics Degree|Computer Science Degree|Mathematics Degree)\b'

# Función para limpiar texto de palabras comunes
def remove_common_words(text):
    return re.sub(common_words, '', text, flags=re.IGNORECASE)

# Función para clasificar las palabras en las categorías
def classify_services(services):
    # Limpiar palabras comunes
    services_cleaned = remove_common_words(services)
    
    # Clasificar términos en categorías
    benefits = re.findall(benefits_keywords, services_cleaned, flags=re.IGNORECASE)
    skills = re.findall(skills_keywords, services_cleaned, flags=re.IGNORECASE)
    tools = re.findall(tools_keywords, services_cleaned, flags=re.IGNORECASE)
    education = re.findall(education_keywords, services_cleaned, flags=re.IGNORECASE)
    
    # Devolver resultados en columnas separadas
    return pd.Series({
        'Beneficios': ', '.join(benefits) if benefits else None,
        'Habilidades': ', '.join(skills) if skills else None,
        'Herramientas': ', '.join(tools) if tools else None,
        'Educación': ', '.join(education) if education else None,
    })

# Aplicar la función a cada fila
df_new = df['Servicios'].apply(classify_services)






   Unnamed: 0                                       Título Ubicación  \
0           0                                 Data Analyst    Madrid   
1           1  Business Intelligence Analyst (all genders)    Madrid   
2           2   Research Analyst (6 month Maternity Cover)    Madrid   
3           3    Senior Program Manager, Analyst Relations    Madrid   
4           4                       [RJ072] | Data Analyst    Madrid   

                                         Descripción  \
0  Una empresa internacional y dinámica busca un ...   
1  An innovative edtech company is seeking a Busi...   
2  An established industry player is seeking a Re...   
3  Join a forward-thinking company at the forefro...   
4  Join a forward-thinking company as a Data Anal...   

                                           Servicios               Sueldo  \
0  ['Tarjeta de restauranteSeguro de saludSeguro ...  EUR 50,000 - 70,000   
1  ['Flexible hybrid work environmentComprehensiv...  EUR 50,000 - 70,000   

In [73]:
df_new['Habilidades']

0                                                    None
1                      Análisis de datos, Data governance
2               Análisis de datos, Visualización de datos
3                                                    None
4               Análisis de datos, Visualización de datos
                              ...                        
1038                  Análisis de datos, Machine learning
1039    Análisis de datos, Machine learning, Visualiza...
1040                                                 None
1041                                                 None
1042                                                 None
Name: Habilidades, Length: 1043, dtype: object

In [72]:
# Diccionario de mapeo para estandarizar las respuestas de habilidades
skills_mapping = {
    r'\b(data analysis|análisis de datos|data science)\b': 'Análisis de datos',
    r'\b(machine learning|aprendizaje automático)\b': 'Machine Learning',
    r'\b(deep learning|aprendizaje profundo)\b': 'Deep Learning',
    r'\b(data visualization|visualización de datos)\b': 'Visualización de datos',
    r'\b(statistical analysis|análisis estadístico)\b': 'Análisis estadístico',
    r'\b(predictive modeling|modelado predictivo)\b': 'Modelado predictivo',
    r'\b(business intelligence|bi)\b': 'Business Intelligence',
    r'\b(reporting|reportes)\b': 'Reporting',
    r'\b(agile methodologies|metodologías ágiles)\b': 'Metodologías Ágiles',
    r'\b(optimización|optimization)\b': 'Optimización',
    r'\b(excel avanzado)\b': 'Excel Avanzado',
    r'\b(habilidades analíticas|analytical skills)\b': 'Habilidades analíticas',
    r'\b(problem solving|resolución de problemas)\b': 'Resolución de problemas',
    r'\b(neural networks|redes neuronales)\b': 'Redes Neuronales'
}

# Función para estandarizar la columna de habilidades
def standardize_skills(text):
    # Verificar si el texto es None antes de proceder
    if text is None:
        return None
    
    # Normalizar cada habilidad según el mapeo
    for pattern, standard_skill in skills_mapping.items():
        text = re.sub(pattern, standard_skill, text, flags=re.IGNORECASE)
    
    # Convertir en lista, eliminar duplicados y ordenar
    skills_list = list(set(text.split(',')))
    skills_list = sorted([skill.strip().capitalize() for skill in skills_list if skill.strip()])
    
    # Unir la lista en una sola cadena de texto
    return ', '.join(skills_list)

df_new['Habilidades'] = df_new['Habilidades'].apply(standardize_skills)


In [74]:
titulos_agrupados = df_new['Habilidades'].value_counts()

In [75]:
titulos_agrupados

Habilidades
Análisis de datos                                                                                         63
Machine learning                                                                                          32
Análisis de datos, Visualización de datos                                                                 21
Análisis de datos, Machine learning                                                                       15
Análisis de datos, Machine learning, Visualización de datos                                               13
                                                                                                          ..
Business intelligence, Machine learning                                                                    1
Análisis de datos, Análisis estadístico, Machine learning, Modelado predictivo, Visualización de datos     1
Análisis de datos, Análisis de datos, Deep learning, Machine learning, Visualización de datos              1
Análisi

In [76]:
# Diccionario de mapeo para estandarizar las respuestas de educación
education_mapping = {
    r'\b(grado)\b': 'Grado',
    r'\b(licenciatura)\b': 'Licenciatura',
    r'\b(engineering degree|ingeniería)\b': 'Ingeniería',
    r'\b(computer science degree)\b': 'Ciencias de la computación',
    r'\b(mathematics degree)\b': 'Matemáticas',
    r'\b(statistics degree)\b': 'Estadística',
    r'\b(ciencias económicas|economics)\b': 'Ciencias económicas'
}

# Función para estandarizar la columna de educación
def standardize_education(text):
    # Verificar si el texto es None antes de proceder
    if text is None:
        return None
    
    # Normalizar cada grado según el mapeo
    for pattern, standard_education in education_mapping.items():
        text = re.sub(pattern, standard_education, text, flags=re.IGNORECASE)
    
    # Convertir en lista, eliminar duplicados y ordenar
    education_list = list(set(text.lower().split(',')))  # Convertir a minúsculas para evitar duplicados por mayúsculas
    education_list = sorted(set([edu.strip().capitalize() for edu in education_list if edu.strip()]))
    
    # Unir la lista en una sola cadena de texto
    return ', '.join(education_list)

# Aplicar la estandarización directamente a la columna "Educación"
df_new['Educación'] = df_new['Educación'].apply(standardize_education)

# Mostrar el DataFrame con la columna estandarizada
print(df_new['Educación'])

0            Licenciatura
1       Bachelor's degree
2                    None
3       Bachelor's degree
4       Bachelor's degree
              ...        
1038                 None
1039      Master's degree
1040                 None
1041                 None
1042                 None
Name: Educación, Length: 1043, dtype: object


In [77]:
# Diccionario de mapeo para estandarizar las respuestas de herramientas
tools_mapping = {
    r'\b(python)\b': 'Python',
    r'\b(excel)\b': 'Excel',
    r'\b(tableau)\b': 'Tableau',
    r'\b(azure)\b': 'Azure',
    r'\b(docker)\b': 'Docker',
    r'\b(snowflake)\b': 'Snowflake',
    r'\b(pyspark)\b': 'PySpark',
    r'\b(scikit-learn|scikit learn)\b': 'Scikit-learn',
    r'\b(databricks)\b': 'Databricks',
    r'\b(google analytics)\b': 'Google Analytics',
    r'\b(adobe analytics)\b': 'Adobe Analytics',
    r'\b(power bi)\b': 'Power BI',
    r'\b(github)\b': 'GitHub',
    r'\b(tensorflow)\b': 'TensorFlow',
    r'\b(keras)\b': 'Keras',
    r'\b(pytorch)\b': 'PyTorch',
    r'\b(jupyter notebooks)\b': 'Jupyter Notebooks',
    r'\b(navision)\b': 'Navision',
    r'\b(refinitiv eikon|eikon)\b': 'Refinitiv Eikon',
    r'\b(bloomberg)\b': 'Bloomberg',
    r'\b(streamlit)\b': 'Streamlit'
}

# Función para estandarizar la columna de herramientas
def standardize_tools(text):
    # Verificar si el texto es None antes de proceder
    if text is None:
        return None
    
    # Normalizar cada herramienta según el mapeo
    for pattern, standard_tool in tools_mapping.items():
        text = re.sub(pattern, standard_tool, text, flags=re.IGNORECASE)
    
    # Convertir en lista, eliminar duplicados y ordenar
    tools_list = list(set(text.lower().split(',')))  # Convertir a minúsculas para evitar duplicados por mayúsculas
    tools_list = sorted(set([tool.strip().capitalize() for tool in tools_list if tool.strip()]))
    
    # Unir la lista en una sola cadena de texto
    return ', '.join(tools_list)

# Aplicar la estandarización directamente a la columna "Herramientas"
df_new['Herramientas'] = df_new['Herramientas'].apply(standardize_tools)

In [81]:
df_new['Herramientas']

0       Excel, Python, R
1                Tableau
2         Excel, Tableau
3                   None
4       Excel, Python, R
              ...       
1038                None
1039       Azure, Python
1040                None
1041                None
1042                None
Name: Herramientas, Length: 1043, dtype: object

In [84]:
df.columns

Index(['Título', 'Puesto', 'sueldo_min', 'sueldo_max', 'Expertise', 'Empresa',
       'Modalidad', 'Sector', 'Ubicación', 'Descripción', 'Servicios',
       'Otro Idioma', 'EntornoTEC'],
      dtype='object')

In [82]:
df_new

,Beneficios,Habilidades,Herramientas,Educación
0,None,None,"Excel, Python, R",Licenciatura
1,None,"Análisis de datos, Data governance",Tableau,Bachelor's degree
2,None,"Análisis de datos, Visualización de datos","Excel, Tableau",None
3,None,None,None,Bachelor's degree
4,None,"Análisis de datos, Visualización de datos","Excel, Python, R",Bachelor's degree
...,...,...,...,...
1038,None,"Análisis de datos, Machine learning",None,None
1039,None,"Análisis de datos, Machine learning, Visualiza...","Azure, Python",Master's degree
1040,None,None,None,None
1041,None,None,None,None


In [85]:
df_final = pd.concat([df, df_new], axis=1)

df_final = df_final.dropna(subset=['Título'])

In [86]:
df_final['Herramientas']

0       Excel, Python, R
1                Tableau
2         Excel, Tableau
3                   None
4       Excel, Python, R
              ...       
1038                None
1039       Azure, Python
1040                None
1041                None
1042                None
Name: Herramientas, Length: 1043, dtype: object

In [87]:
df_final = df_final.drop_duplicates()

In [88]:
df_final.columns

Index(['Título', 'Puesto', 'sueldo_min', 'sueldo_max', 'Expertise', 'Empresa',
       'Modalidad', 'Sector', 'Ubicación', 'Descripción', 'Servicios',
       'Otro Idioma', 'EntornoTEC', 'Beneficios', 'Habilidades',
       'Herramientas', 'Educación'],
      dtype='object')

In [89]:

df_final.to_csv("Datos_definitivos.csv", index = False)
# Cargar los datos desde un archivo CSV
data = pd.read_csv('Datos_definitivos.csv')

# Verificar las primeras filas de los datos
print(data.head())

# Preparar los datos para el clustering (selecciona las características relevantes)
X = data[['Puesto', 'Expertise', 'Sector']]

# Convertir variables categóricas a numéricas
X['Puesto'] = X['Puesto'].astype('category').cat.codes
X['Expertise'] = X['Expertise'].astype('category').cat.codes
X['Sector'] = X['Sector'].astype('category').cat.codes

# Aplicar K-means para segmentar los datos
kmeans = KMeans(n_clusters=3, random_state=0)
data['Segmento'] = kmeans.fit_predict(X)

# Visualizar la cantidad de ofertas por segmento
segment_counts = data['Segmento'].value_counts().reset_index()
segment_counts.columns = ['Segmento', 'Cantidad']

# Gráfico de barras mostrando la cantidad de ofertas por segmento
fig_barras = go.Figure(data=[
    go.Bar(x=segment_counts['Segmento'], y=segment_counts['Cantidad'], 
           marker=dict(color='royalblue'))
])

# Configurar el layout del gráfico de barras
fig_barras.update_layout(
    title='Cantidad de Ofertas por Segmento',
    xaxis_title='Segmento',
    yaxis_title='Cantidad de Ofertas'
)

# Guardar el gráfico como archivo HTML y abrirlo en el navegador
pio.write_html(fig_barras, 'grafico_barras.html')
import webbrowser
webbrowser.open('grafico_barras.html')

# Gráfico de dispersión mostrando los diferentes segmentos
fig_dispersion = go.Figure()

# Agregar trazas para cada segmento
for segmento in data['Segmento'].unique():
    seg_data = data[data['Segmento'] == segmento]
    fig_dispersion.add_trace(go.Scatter(
        x=seg_data['Puesto'], 
        y=seg_data['Expertise'], 
        mode='markers',
        name=f'Segmento {segmento}',
        text=seg_data['Descripción'],  # Mostrar descripción al pasar el ratón
        marker=dict(size=10)
    ))

# Configurar el layout del gráfico de dispersión
fig_dispersion.update_layout(
    title='Resultados del Análisis de Segmentación',
    xaxis_title='Puesto',
    yaxis_title='Expertise',
    legend_title='Segmentos',
    hovermode='closest'
)

# Guardar el gráfico de dispersión como archivo HTML y abrirlo en el navegador
pio.write_html(fig_dispersion, 'grafico_dispersion.html')
webbrowser.open('grafico_dispersion.html')
# Asegurarte de que el DataFrame tiene los nombres de columnas correctos
df_final.columns = [
    "Título", "Puesto", 'sueldo_min','sueldo_max',"Expertise", "Empresa", "Modalidad", "Sector", 
    "Ubicación", "Descripción", "Servicios", "Otro_Idioma", "EntornoTEC", 
    "Beneficios", "Habilidades", "Herramientas", "Educación"
]


                                     Título                 Puesto  \
0                              data analyst           Data Analyst   
1             business intelligence analyst  Business Intelligence   
2                          research analyst           Data Analyst   
3  senior program manager analyst relations           Data Analyst   
4                              data analyst           Data Analyst   

   sueldo_min  sueldo_max Expertise Empresa Modalidad Sector Ubicación  \
0       50000       70000   Regular     NaN       NaN    NaN    Madrid   
1       50000       70000   Regular     NaN       NaN    NaN    Madrid   
2       30000       50000   Regular     NaN       NaN    NaN    Madrid   
3       50000       70000    Senior     NaN       NaN    NaN    Madrid   
4       50000       70000   Regular     NaN       NaN    NaN    Madrid   

                                         Descripción  \
0  Una empresa internacional y dinámica busca un ...   
1  An innovative edtec

/var/folders/5s/6q7y_kyj6zl53g0f3jycmzsr0000gn/T/ipykernel_79545/3307121618.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5s/6q7y_kyj6zl53g0f3jycmzsr0000gn/T/ipykernel_79545/3307121618.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5s/6q7y_kyj6zl53g0f3jycmzsr0000gn/T/ipykernel_79545/3307121618.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [90]:
df_final.columns

Index(['Título', 'Puesto', 'sueldo_min', 'sueldo_max', 'Expertise', 'Empresa',
       'Modalidad', 'Sector', 'Ubicación', 'Descripción', 'Servicios',
       'Otro_Idioma', 'EntornoTEC', 'Beneficios', 'Habilidades',
       'Herramientas', 'Educación'],
      dtype='object')

In [91]:

# Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')
cursor = conn.cursor()
# Eliminamos y creamos la tabla
cursor.execute("DROP TABLE IF EXISTS bbdd_ofertas")
cursor.execute('''
    CREATE TABLE bbdd_ofertas (
        Título TEXT,
        Puesto TEXT,
        Expertise TEXT,
        Empresa TEXT,
        Modalidad TEXT,
        sueldo_min TEXT,
        sueldo_max TEXT,
        Sector TEXT,
        Ubicación TEXT,
        Descripción TEXT,
        Servicios TEXT,
        Otro_Idioma TEXT,
        EntornoTEC TEXT,
        Beneficios TEXT,
        Habilidades TEXT,
        Herramientas TEXT,
        Educación TEXT
    )
''')


# Asegúrate de que el DataFrame tiene los nombres de columnas correctos
print(df_final.columns)  # Verificar los nombres de las columnas

# Insertar el DataFrame en la tabla de SQLite
df_final.to_sql('bbdd_ofertas', conn, if_exists='append', index=False)

# Verificar que los datos se hayan guardado correctamente
df_check = pd.read_sql("SELECT * FROM bbdd_ofertas LIMIT 5", conn)
print(df_check)

# Cerrar la conexión
conn.close()


# Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')

# Leer los datos de la tabla 'job_data' en un DataFrame de pandas
query = "SELECT * FROM bbdd_ofertas LIMIT 5"
df_check = pd.read_sql(query, conn)

# Mostrar el DataFrame para verificar que los datos se hayan guardado
print(df_check)

# Cerrar la conexión
conn.close()
 # Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')

# Consultar los datos donde la ubicación es "Madrid"
query = "SELECT * FROM bbdd_ofertas WHERE Ubicación = 'Madrid'"
df_madrid = pd.read_sql(query, conn)

# Mostrar el DataFrame con los registros de Madrid
print(df_madrid)

# Cerrar la conexión
conn.close()

Index(['Título', 'Puesto', 'sueldo_min', 'sueldo_max', 'Expertise', 'Empresa',
       'Modalidad', 'Sector', 'Ubicación', 'Descripción', 'Servicios',
       'Otro_Idioma', 'EntornoTEC', 'Beneficios', 'Habilidades',
       'Herramientas', 'Educación'],
      dtype='object')
                                     Título                 Puesto Expertise  \
0                              data analyst           Data Analyst   Regular   
1             business intelligence analyst  Business Intelligence   Regular   
2                          research analyst           Data Analyst   Regular   
3  senior program manager analyst relations           Data Analyst    Senior   
4                              data analyst           Data Analyst   Regular   

  Empresa Modalidad sueldo_min sueldo_max Sector Ubicación  \
0    None      None      50000      70000   None    Madrid   
1    None      None      50000      70000   None    Madrid   
2    None      None      30000      50000   None    Madrid 

In [92]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 749 entries, 0 to 1037
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Título        749 non-null    object
 1   Puesto        749 non-null    object
 2   sueldo_min    749 non-null    int64 
 3   sueldo_max    749 non-null    int64 
 4   Expertise     749 non-null    object
 5   Empresa       16 non-null     object
 6   Modalidad     9 non-null      object
 7   Sector        70 non-null     object
 8   Ubicación     749 non-null    object
 9   Descripción   749 non-null    object
 10  Servicios     749 non-null    object
 11  Otro_Idioma   14 non-null     object
 12  EntornoTEC    59 non-null     object
 13  Beneficios    46 non-null     object
 14  Habilidades   261 non-null    object
 15  Herramientas  465 non-null    object
 16  Educación     282 non-null    object
dtypes: int64(2), object(15)
memory usage: 105.3+ KB


In [93]:
df_final['Puesto'].value_counts()

Puesto
Data Analyst                 344
Data Engineer                146
Data Scientist               120
Other                         70
Data Specialist               19
Machine Learning Engineer     19
Business Intelligence         11
Consultant                     6
Data Analytics                 6
Data Developer                 4
Project Manager                2
Data Expert                    1
AI Innovation Architect        1
Name: count, dtype: int64

In [94]:
df_final['Herramientas'].value_counts()

Herramientas
Python                                           58
Excel                                            57
R                                                46
Python, R                                        25
Python, Tableau                                  21
                                                 ..
Azure, Docker, Keras, Python, Scikit-learn        1
Azure, Docker, Keras, Python, R, Scikit-learn     1
Databricks                                        1
Docker, Python, R                                 1
Docker, Keras, Python, Tensorflow                 1
Name: count, Length: 98, dtype: int64